<a href="https://colab.research.google.com/github/ImineAmazigh/MadLibs/blob/main/PricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#these are just libraries, that will be used, if you want to install more go to terminal or tell me
import tensorflow as tf
import optuna
import pandas as pd
import numpy as np
import re

from google.colab import files

In [ ]:
df = pd.read_csv('/content/train_car.csv')
print(df)
id_train = df['id'].to_numpy(dtype=np.int32)
print(id_train)
model_train = df['model'].to_numpy(dtype=str)
print(model_train)
mileage_km_train = df['mileage_km'].to_numpy(dtype = np.int32)
print(mileage_km_train)
year_train = df['year'].to_numpy(dtype=np.float16)
print(year_train)
transmission_train = df['transmission'].to_numpy(dtype=str)
print(transmission_train)
fuel_type_train = df['fuel_type'].to_numpy(dtype=str)
print(fuel_type_train)
listing_date_train = df['listing_date']
to_date_var = pd.to_datetime(listing_date_train)
df['listing_date'] = to_date_var.dt.year + (to_date_var.dt.dayofyear - 1)/365
listing_date_train = df['listing_date'].to_numpy(dtype=np.float64)
print(listing_date_train)


float16


In [ ]:
lrBest = 0.01247995137227659
echosBest = 19
n1Best = 32
n2Best = 32
global model
model = tf.keras.Sequential([tf.keras.Input(shape= (7,)), # Changed shape to match matrixHolder.T
                               tf.keras.layers.Dense(n1Best, activation='relu'),
                               tf.keras.layers.Dense(n2Best,activation='relu'),
                               tf.keras.layers.Dense(1)])
model.compile(optimizer='adamw',
                metrics=[tf.keras.metrics.RootMeanSquaredError()],
                loss='mse')

  # Use the pre-split training data
X_numeric = np.asarray(X_train, dtype=np.float32)
y_numeric = np.asarray(y_train, dtype=np.float32)

model.fit(X_numeric, y_numeric, epochs=echosBest, verbose=0)

predictions = model.evaluate(X_test,y_test)


60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 963147857920.0000 - root_mean_squared_error: 915057.8125 
here start the evaluating 




